# Pose linear metrics — stats & per-metric barplots

Loads linear metric CSVs from:
`data/processed_data/<session>/linear_metrics/*.csv`

Produces:
- `data/stats_results/<session>_pose_lin_table.tex`
- `figs/pose_linear/<session>/<metric>.svg` (one SVG per modeled metric)


## 1. Imports and Load Data

In [1]:
from pathlib import Path
import sys
sys.path.append(str(Path("utils").resolve()))
from utils.stats_utils import discover_linear_files, load_session_csvs, build_table_with_emmeans

ROOT = Path("data/processed_data")
OUT_TABLE_DIR = Path("tables/")
FIGS_ROOT = Path("figs/pose_linear")
OUT_TABLE_DIR.mkdir(parents=True, exist_ok=True)
FIGS_ROOT.mkdir(parents=True, exist_ok=True)

sessions = discover_linear_files(ROOT)
print("Found sessions:", sessions.keys())


Found sessions: dict_keys(['baseline', 'experimental'])


## 2. Run Mixed Effects Models and Output Tables

In [2]:
# process per-session, per-method (recommended, without t_start_frame/t_end_frame)
sessions = discover_linear_files(ROOT)
for session_name, files in sessions.items():
    print(f"\n=== SESSION: {session_name} ===")
    df_all = load_session_csvs(files)
    if df_all.empty:
        print("No files/data for", session_name); 
        continue

    # drop frame columns 
    df_all = df_all.drop(columns=[c for c in ["t_start_frame","t_end_frame"] if c in df_all.columns])

    # detect method by filename or `_source_file` column
    for method in ("original", "procrustes"):
        df_m = df_all[df_all["_source_file"].str.contains(method, case=False, na=False)]
        if df_m.empty:
            print(f"  [skip] no {method} files")
            continue

        out_tex = OUT_TABLE_DIR / f"{session_name}_{method}_pose_lin_table.tex"
        figs_dir = FIGS_ROOT / session_name / method

        modeled, skipped = build_table_with_emmeans(df_m, out_tex, figs_dir)
        print(f"  [{method}] modeled={modeled}, skipped={skipped}")



=== SESSION: baseline ===


R callback write-console: boundary (singular) fit: see help('isSingular')
  
R callback write-console: boundary (singular) fit: see help('isSingular')
  
R callback write-console: boundary (singular) fit: see help('isSingular')
  
R callback write-console: boundary (singular) fit: see help('isSingular')
  
R callback write-console: boundary (singular) fit: see help('isSingular')
  
R callback write-console: boundary (singular) fit: see help('isSingular')
  


[OK] wrote tables/baseline_original_pose_lin_table.tex | modeled=36, skipped=20
  [original] modeled=36, skipped=20


R callback write-console: boundary (singular) fit: see help('isSingular')
  
R callback write-console: boundary (singular) fit: see help('isSingular')
  
R callback write-console: boundary (singular) fit: see help('isSingular')
  
R callback write-console: boundary (singular) fit: see help('isSingular')
  
R callback write-console: boundary (singular) fit: see help('isSingular')
  
R callback write-console: boundary (singular) fit: see help('isSingular')
  
R callback write-console: boundary (singular) fit: see help('isSingular')
  
R callback write-console: boundary (singular) fit: see help('isSingular')
  
R callback write-console: boundary (singular) fit: see help('isSingular')
  
R callback write-console: boundary (singular) fit: see help('isSingular')
  
R callback write-console: boundary (singular) fit: see help('isSingular')
  
R callback write-console: boundary (singular) fit: see help('isSingular')
  
R callback write-console: boundary (singular) fit: see help('isSingular')
  

[OK] wrote tables/baseline_procrustes_pose_lin_table.tex | modeled=44, skipped=12
  [procrustes] modeled=44, skipped=12

=== SESSION: experimental ===
[OK] wrote tables/experimental_original_pose_lin_table.tex | modeled=36, skipped=20
  [original] modeled=36, skipped=20


R callback write-console: boundary (singular) fit: see help('isSingular')
  
R callback write-console: boundary (singular) fit: see help('isSingular')
  
R callback write-console: boundary (singular) fit: see help('isSingular')
  


[OK] wrote tables/experimental_procrustes_pose_lin_table.tex | modeled=44, skipped=12
  [procrustes] modeled=44, skipped=12


## 3. Plot Example Figures

In [3]:
from pathlib import Path
from IPython.display import SVG, display, clear_output
import ipywidgets as widgets

FIGS_ROOT = Path("figs/pose_linear")

# Collect session → method → svg mapping
fig_map = {}
for session in FIGS_ROOT.iterdir():
    if session.is_dir():
        fig_map[session.name] = {}
        for method in ("original", "procrustes"):
            method_dir = session / method
            if method_dir.exists():
                svgs = sorted(method_dir.glob("*.svg"))
                fig_map[session.name][method] = {f.stem: f for f in svgs}

# Widgets
session_dd = widgets.Dropdown(
    options=sorted(fig_map.keys()),
    description="Session:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="40%")
)

method_dd = widgets.Dropdown(
    options=[],
    description="Method:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="40%")
)

metric_dd = widgets.Dropdown(
    options=[],
    description="Metric:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width="60%")
)

output = widgets.Output()

def update_methods(*args):
    session = session_dd.value
    if session and session in fig_map:
        method_dd.options = sorted(fig_map[session].keys())
        method_dd.value = next(iter(method_dd.options), None)

def update_metrics(*args):
    session, method = session_dd.value, method_dd.value
    if session and method and method in fig_map[session]:
        metric_dd.options = sorted(fig_map[session][method].keys())
        metric_dd.value = next(iter(metric_dd.options), None)

def show_svg(*args):
    output.clear_output()
    session, method, metric = session_dd.value, method_dd.value, metric_dd.value
    if session and method and metric:
        svg_path = fig_map[session][method][metric]
        with output:
            display(SVG(filename=str(svg_path)))

# Wire callbacks
session_dd.observe(update_methods, names='value')
method_dd.observe(update_metrics, names='value')
metric_dd.observe(show_svg, names='value')

# Initialize
update_methods()
update_metrics()
show_svg()

# Display controls
widgets.VBox([session_dd, method_dd, metric_dd, output])
